<a href="https://www.kaggle.com/code/vtrackstar/data-analysis-project-olympic-t-f?scriptVersionId=194867208" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Ask
* The goal of this project is to predict future Olympic medalists in track and field events using over 100 years of historical data. 
* By analyzing trends in athlete performance, country dominance, and changes in event characteristics, I will develop machine learning models to forecast potential medalists.

## Prepare and Process

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

file_path = '/kaggle/input/olympic-track-field-results/results.csv'

# skip bad lines
try:
    df = pd.read_csv(file_path, on_bad_lines='skip')
except Exception as e:
    print(f"An error occurred: {e}")

# Display the first few rows to verify successful loading
print(df.head())

# Show distinct values in the "Event" column
if 'Event' in df.columns:
    distinct_events = df['Event'].unique()
    print("Distinct values in 'Event' column:")
    print(distinct_events)
else:
    print("'Event' column not found in the DataFrame.")
    
# Define keywords to filter out
keywords = ["Relay", "Decathlon", "Heptathlon"]

# Create a regex pattern to match any of the keywords
pattern = '|'.join(keywords)

# Drop rows where 'Event' column contains any of the keywords
df_cleaned = df[~df['Event'].str.contains(pattern, case=False, na=False)]

# Display the first few rows of the cleaned DataFrame
print(df_cleaned.head())

  Gender       Event Location  Year Medal                   Name Nationality  \
0      M  10000M Men      Rio  2016     G          Mohamed FARAH         USA   
1      M  10000M Men      Rio  2016     S  Paul Kipngetich TANUI         KEN   
2      M  10000M Men      Rio  2016     B           Tamirat TOLA         ETH   
3      M  10000M Men  Beijing  2008     G        Kenenisa BEKELE         ETH   
4      M  10000M Men  Beijing  2008     S         Sileshi SIHINE         ETH   

     Result  
0  25:05.17  
1  27:05.64  
2  27:06.26  
3  27:01.17  
4  27:02.77  
Distinct values in 'Event' column:
['10000M Men' '100M Men' '110M Hurdles Men' '1500M Men' '200M Men'
 '20Km Race Walk Men' '3000M Steeplechase Men' '400M Hurdles Men'
 '400M Men' '4X100M Relay Men' '4X400M Relay Men' '5000M Men'
 '50Km Race Walk Men' '800M Men' 'Decathlon Men' 'Discus Throw Men'
 'Hammer Throw Men' 'High Jump Men' 'Javelin Throw Men' 'Long Jump Men'
 'Marathon Men' 'Pole Vault Men' 'Shot Put Men' 'Triple Jump Men'

In [2]:
file_path1 = '/kaggle/input/paris2024-olympics-country-level-data/Paris2024_olympics_country_data.csv'
df1 = pd.read_csv(file_path1)

# Display the first few rows of the DataFrame to understand its structure
print(df1.head())

# Add Country column from Olympic Medal Dataset to Olympic Track and Field Dataset
merged_df = pd.merge(df_cleaned, df1[['Country Code', 'Country']], left_on='Nationality', right_on='Country Code', how='left')

# Drop the extra 'country_code' column if you only need 'country'
merged_df = merged_df.drop(columns='Country Code')

print(merged_df.head())

# Function to convert "Result" column time formats to seconds
def convert_to_seconds(Result):
    if isinstance(Result, str):
        # Remove unwanted text like " est" or any other text after a space
        Result = Result.split()[0]
        
        # Skip entries with a dash, assuming they are distances and not times
        if '-' in Result:
            return np.nan
        
        # Handle the case where the format is '1h19' or similar
        if 'h' in Result:
            hours, rest = Result.split('h')
            minutes, seconds = 0, 0
            if ':' in rest:
                parts = rest.split(':')
                if len(parts) == 2:
                    minutes = float(parts[0])
                    seconds = float(parts[1])
            else:
                minutes = float(rest)
            total_seconds = float(hours) * 3600 + minutes * 60 + seconds
            return round(total_seconds, 2)
        
        if ':' in Result:
            # Convert HH:MM:SS or MM:SS format to seconds
            parts = Result.split(':')
            if len(parts) == 3:  # HH:MM:SS format
                hours = float(parts[0])
                minutes = float(parts[1])
                seconds = float(parts[2])
                total_seconds = hours * 3600 + minutes * 60 + seconds
            elif len(parts) == 2:  # MM:SS format
                minutes = float(parts[0])
                seconds = float(parts[1])
                total_seconds = minutes * 60 + seconds
            return round(total_seconds, 2)
        
        # Directly return if it's already in seconds, formatting as SS.XX
        try:
            return round(float(Result), 2)
        except ValueError:
            return np.nan
    
    return np.nan

# Add the 'Result(S)' column to merged_df
merged_df['Result(S)'] = merged_df['Result'].apply(convert_to_seconds)

# Remove rows where 'Result(S)' is NaN (which indicates invalid or non-time data)
merged_df = merged_df.dropna(subset=['Result(S)'])

# Display the updated DataFrame to confirm the new column
print(merged_df[['Result', 'Result(S)']].head())

   Unnamed: 0         Country Country Code  Number of athletes  Gold medals  \
0           0     Afghanistan          AFG                   6            0   
1           1         Albania          ALB                   8            0   
2           2         Algeria          DZA                  45            2   
3           3  American Samoa          ASM                   2            0   
4           4         Andorra          AND                   2            0   

   Silver medals  Bronze medals  Total medals           GDP  GDP per capita  \
0              0              0             0  1.450216e+10      352.603733   
1              0              2             2  1.891638e+10     6810.114041   
2              0              1             3  2.255603e+11     5023.252932   
3              0              0             0  8.710000e+08    19673.390102   
4              0              0             0  3.380602e+09    42350.697069   

   Population  Life expectancy  Democracy  Gender 

In [3]:
# Mapping abbreviations to full names
medal_mapping = {'G': 'Gold', 'S': 'Silver', 'B': 'Bronze'}

# Apply the mapping to convert abbreviations to full names
merged_df['Medal'] = merged_df['Medal'].map(medal_mapping)

# Count the number of each type of medal by country and event
medal_counts = merged_df.groupby(['Country', 'Event'])['Medal'].value_counts().unstack(fill_value=0).reset_index()

# Rename columns for clarity
medal_counts.columns.name = None
medal_counts.columns = ['Country', 'Event', 'Gold', 'Silver', 'Bronze']

# Define the event groups
sprint_events = ['100M Men', '100M Women', '200M Men', '200M Women', '110M Hurdles Men', '100M Hurdles Women', '400M Men', '400M Women', '400M Hurdles Men', '400M Hurdles Women']
mid_events = ['800M Men', '800M Women', '3000M Steeplechase Men', '3000M Steeplechase Women', '1500M Men', '1500M Women']
distance_events = ['5000M Men', '5000M Women', '10000m Men', '10000M Women', 'Marathon Men', 'Marathon Women']
throw_events = ['Hammer Throw Men', 'Hammer Throw Women', 'Discus Throw Men', 'Discus Throw Women', 'Shot Put Men', 'Shot Put Women', 'Javelin Throw Men', 'Javelin Throw Women']
jump_events = ['Triple Jump Men', 'Triple Jump Women', 'Long Jump Men', 'Long Jump Women', 'High Jump Men', 'High Jump Women', 'Pole Vault Men', 'Pole Vault Women']
walk_events = ['20Km Race Walk Men', '20Km Race Walk Women']
multi_events = ['Decathlon Men', 'Heptathlon Women']
relay_events = ['4X100M Relay Men', '4X100M Relay Women', '4X400M Relay Men', '4X400M Relay Women']

# Filtering rows by Event Group
sprint = medal_counts[medal_counts['Event'].isin(sprint_events)]
mid = medal_counts[medal_counts['Event'].isin(mid_events)]
distance = medal_counts[medal_counts['Event'].isin(distance_events)]
throw = medal_counts[medal_counts['Event'].isin(throw_events)]
jump = medal_counts[medal_counts['Event'].isin(jump_events)]
walk = medal_counts[medal_counts['Event'].isin(walk_events)]
multi = medal_counts[medal_counts['Event'].isin(multi_events)]
relay = medal_counts[medal_counts['Event'].isin(relay_events)]

# Add a Total Medals column
sprint['Total Medals'] = sprint['Gold'] + sprint['Silver'] + sprint['Bronze']
mid['Total Medals'] = mid['Gold'] + mid['Silver'] + mid['Bronze']
distance['Total Medals'] = distance['Gold'] + distance['Silver'] + distance['Bronze']
throw['Total Medals'] = throw['Gold'] + throw['Silver'] + throw['Bronze']
jump['Total Medals'] = jump['Gold'] + jump['Silver'] + jump['Bronze']
walk['Total Medals'] = walk['Gold'] + walk['Silver'] + walk['Bronze']
multi['Total Medals'] = multi['Gold'] + multi['Silver'] + multi['Bronze']
relay['Total Medals'] = relay['Gold'] + relay['Silver'] + relay['Bronze']

# Sort the DataFrame by Total Medals in descending order
sprint = sprint.sort_values(by='Total Medals', ascending=False)

print("Sprint Domination - Event Medal Counts by Country:")
sprint.head()

Sprint Domination - Event Medal Counts by Country:


/tmp/ipykernel_18/811552738.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sprint['Total Medals'] = sprint['Gold'] + sprint['Silver'] + sprint['Bronze']
/tmp/ipykernel_18/811552738.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mid['Total Medals'] = mid['Gold'] + mid['Silver'] + mid['Bronze']
/tmp/ipykernel_18/811552738.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

,Country,Event,Gold,Silver,Bronze,Total Medals
398,United States,110M Hurdles Men,14,18,19,51
407,United States,400M Men,11,18,12,41
401,United States,200M Men,8,16,15,39
405,United States,400M Hurdles Men,8,18,11,37
396,United States,100M Men,8,15,13,36


In [4]:
# Sort the DataFrame by Total Medals in descending order
mid = mid.sort_values(by='Total Medals', ascending=False)

print("Middle Distance Domination - Event Medal Counts by Country:")
mid.head()

Middle Distance Domination - Event Medal Counts by Country:


,Country,Event,Gold,Silver,Bronze,Total Medals
411,United States,800M Men,8,8,4,20
251,Kenya,3000M Steeplechase Men,4,10,6,20
399,United States,1500M Men,3,3,7,13
156,Great Britain,1500M Men,3,5,4,12
257,Kenya,800M Men,5,4,2,11


In [5]:
# Sort the DataFrame by Total Medals in descending order
distance = distance.sort_values(by='Total Medals', ascending=False)

print("Long Distance Domination - Event Medal Counts by Country:")
distance.head()

Long Distance Domination - Event Medal Counts by Country:


,Country,Event,Gold,Silver,Bronze,Total Medals
122,Finland,5000M Men,2,7,4,13
422,United States,Marathon Men,5,3,3,11
108,Ethiopia,10000M Women,3,5,2,10
114,Ethiopia,Marathon Men,3,4,1,8
113,Ethiopia,5000M Women,5,3,0,8


In [6]:
# Sort the DataFrame by Total Medals in descending order
throw = throw.sort_values(by='Total Medals', ascending=False)

print("Throw Domination - Event Medal Counts by Country:")
throw.head()

Throw Domination - Event Medal Counts by Country:


,Country,Event,Gold,Silver,Bronze,Total Medals
426,United States,Shot Put Men,10,14,14,38
413,United States,Discus Throw Men,8,10,6,24
415,United States,Hammer Throw Men,4,5,5,14
125,Finland,Javelin Throw Men,5,4,4,13
188,Hungary,Hammer Throw Men,1,3,2,6


In [7]:
# Sort the DataFrame by Total Medals in descending order
jump = jump.sort_values(by='Total Medals', ascending=False)

print("Jump Domination - Event Medal Counts by Country:")
jump.head()

Jump Domination - Event Medal Counts by Country:


,Country,Event,Gold,Silver,Bronze,Total Medals
420,United States,Long Jump Men,6,16,12,34
424,United States,Pole Vault Men,6,13,10,29
416,United States,High Jump Men,5,8,9,22
428,United States,Triple Jump Men,2,7,5,14
421,United States,Long Jump Women,3,2,2,7


In [8]:
# Sort the DataFrame by Total Medals in descending order
walk = walk.sort_values(by='Total Medals', ascending=False)

print("Racewalk Domination - Event Medal Counts by Country:")
walk.head()

Racewalk Domination - Event Medal Counts by Country:


,Country,Event,Gold,Silver,Bronze,Total Medals
263,Mexico,20Km Race Walk Men,1,2,3,6
68,China,20Km Race Walk Women,2,3,0,5
210,Italy,20Km Race Walk Men,3,2,0,5
9,Australia,20Km Race Walk Men,3,0,1,4
67,China,20Km Race Walk Men,1,2,1,4


## Analyze

## Share
Interactive Tableau Dashboard Coming Soon
### 400M Olympic Regression Analysis Interpretation
* Men's 400m Regression Analysis:
    * Mean Absolute Error (MAE): **0.966 seconds**
    * On average, the model's predictions are off by about 0.966 seconds from the actual race times. This reflects the model's accuracy in predicting men's 400m times.
    * R-squared (R²): **0.709**
    * The model explains approximately 70.9% of the variance in men's 400m times, indicating a strong correlation between the year of the race and the time, and a good fit for the data.
* Women's 400m Regression Analysis:
    * Mean Absolute Error (MAE): **0.739 seconds**
    * The model's predictions are, on average, about 0.739 seconds off from the actual race times. This suggests that the model is fairly accurate for women's 400m times.
    * R-squared (R²): **-0.002**
    * The model's fit is poor, with an R-squared value slightly below zero, indicating that it explains almost none of the variance in women's 400m times. The model does not effectively capture the relationship between the year of the race and the time.

## Act
### 400M Regression Model Insights
* The regression model effectively captures the trend in men's 400m times over the years, with a high R-squared value (0.709). This suggests that improvements in race times over time are well-explained by the model. 
    * To build on this, future analyses could incorporate additional variables such as technological advancements or training techniques to refine predictions further and explore underlying factors contributing to the observed trends.
* For Women's 400m, the regression model's performance for women's 400m times is less effective, as indicated by the negative R-squared value (-0.002). This suggests that the model does not explain the variability in race times well. 
    * To address this, future analysis can include more features or exploring other modeling techniques. Additional factors, such as changes in training practices, coaching strategies, or athlete nutrition over the years, may provide better insights into trends in women's 400m times.